In [2]:
import pandas as pd
import numpy as np
import os
import warnings
from scipy.sparse import csr_matrix
from sklearn.preprocessing import StandardScaler
warnings.filterwarnings("ignore")
os.chdir('E:\\movieLens dataset')

In [3]:
data = pd.read_csv('u.data', sep='\t', names=['userId', 'movieId', 'rating', 'timestamp'])
movie = pd.read_csv('u.item', sep='\|', header=None)
movie = movie[[0, 1, 2]]
movie.columns = ['movieId', 'title', 'date']

In [4]:
movie[movie['movieId'] == 1]

,movieId,title,date
0,1,Toy Story (1995),01-Jan-1995


In [5]:
temp = data.groupby('movieId').count()['rating'].sort_values(ascending=False).to_frame()
temp1 = data.groupby('userId').count()['rating'].sort_values(ascending=False).to_frame()
less3 = temp[temp['rating']<3]
data = data[~data['movieId'].isin(less3.index)]
pivot = pd.pivot_table(data=data, index='userId', columns='movieId', values='rating')
pivot['userId'] = pivot.index

In [6]:
pivot = pivot.drop(columns='userId')
pivot

movieId,1,2,3,4,5,6,7,8,9,10,...,1615,1620,1622,1623,1628,1639,1643,1652,1658,1664
userId,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
940,NaN,NaN,NaN,2.0,NaN,NaN,4.0,5.0,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
941,5.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
#Высчитывание среднего и вычитание из значений рейтингов
users_mean_rating = np.array(pivot.mean(axis=1))
pivot_demeaned = pivot.sub(pivot.mean(axis=1), axis=0)
pivot_demeaned = pivot_demeaned.fillna(0).values
pivot_demeaned = csr_matrix(pivot_demeaned)
print(pivot_demeaned)

  (0, 0)	1.3897058823529411
  (0, 1)	-0.6102941176470589
  (0, 2)	0.3897058823529411
  (0, 3)	-0.6102941176470589
  (0, 4)	-0.6102941176470589
  (0, 5)	1.3897058823529411
  (0, 6)	0.3897058823529411
  (0, 7)	-2.610294117647059
  (0, 8)	1.3897058823529411
  (0, 9)	-0.6102941176470589
  (0, 10)	-1.6102941176470589
  (0, 11)	1.3897058823529411
  (0, 12)	1.3897058823529411
  (0, 13)	1.3897058823529411
  (0, 14)	1.3897058823529411
  (0, 15)	1.3897058823529411
  (0, 16)	-0.6102941176470589
  (0, 17)	0.3897058823529411
  (0, 18)	1.3897058823529411
  (0, 19)	0.3897058823529411
  (0, 20)	-2.610294117647059
  (0, 21)	0.3897058823529411
  (0, 22)	0.3897058823529411
  (0, 23)	-0.6102941176470589
  (0, 24)	0.3897058823529411
  :	:
  (942, 734)	0.5892857142857144
  (942, 751)	-1.4107142857142856
  (942, 758)	0.5892857142857144
  (942, 760)	-0.4107142857142856
  (942, 779)	-1.4107142857142856
  (942, 788)	-0.4107142857142856
  (942, 790)	-0.4107142857142856
  (942, 802)	0.5892857142857144
  (942, 809

In [33]:
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import GridSearchCV
for i in [2, 5, 7, 10, 15, 30, 50, 100, 200, 500]:
    svd = TruncatedSVD(n_components=i, random_state=17)
    matrix = svd.fit_transform(pivot_demeaned)
    U = matrix / svd.singular_values_
    Sigma = np.diag(svd.singular_values_)
    Vt = svd.components_
    all_user_predicted_ratings = np.dot(np.dot(U, Sigma), Vt) + users_mean_rating.reshape(-1, 1)
    preds_df = pd.DataFrame(all_user_predicted_ratings, columns = pivot.columns)
    preds_sps = preds_df.values
    pivot_sps = pivot.values
    pivot_sps_1 = pivot_sps[~np.isnan(pivot_sps)]
    preds_sps_1 = preds_sps[~np.isnan(pivot_sps)]
    err = mse(pivot_sps_1, preds_sps_1)
    rmse = sqrt(err)
    print(f'Компонент {i} - rmse = {rmse}')

Компонент 2 - rmse = 0.9650751769941499
Компонент 5 - rmse = 0.9370272559409463
Компонент 7 - rmse = 0.92157643330745
Компонент 10 - rmse = 0.9026528348724762
Компонент 15 - rmse = 0.8746532785201601
Компонент 30 - rmse = 0.8042597767727958
Компонент 50 - rmse = 0.7263971294028894
Компонент 100 - rmse = 0.5748958654226871
Компонент 200 - rmse = 0.3695658040827465
Компонент 500 - rmse = 0.08907266816364375


In [9]:
U = matrix / svd.singular_values_
Sigma = np.diag(svd.singular_values_)
Vt = svd.components_

In [10]:
#Обратное умножение матриц и сумма среднего
all_user_predicted_ratings = np.dot(np.dot(U, Sigma), Vt) + users_mean_rating.reshape(-1, 1)
#  + users_mean_rating.reshape(-1, 1)

In [34]:
# all_user_predicted_ratings = scaler.inverse_transform(all_user_predicted_ratings)
preds_df = pd.DataFrame(all_user_predicted_ratings, columns = pivot.columns)
preds_df

movieId,1,2,3,4,5,6,7,8,9,10,...,1615,1620,1622,1623,1628,1639,1643,1652,1658,1664
0,5.000438,2.994650,4.014497,2.999610,3.018607,4.994125,3.995046,0.994580,5.003498,3.001814,...,3.590281,3.618102,3.615499,3.606701,3.604830,3.598559,3.606239,3.596431,3.609437,3.613500
1,4.038045,3.677415,3.614962,3.648138,3.606396,3.640452,3.704791,3.677581,3.761565,2.046995,...,3.747689,3.710141,3.684097,3.715421,3.684813,3.682498,3.730153,3.708334,3.704153,3.683676
2,2.770314,2.807279,2.723462,2.891073,2.794350,2.686434,2.799081,2.783395,2.802460,2.799570,...,2.816664,2.785061,2.780528,2.804023,2.784609,2.800038,2.802477,2.842480,2.800210,2.830675
3,4.327614,4.308382,4.284135,4.337072,4.346235,4.354183,4.318960,4.354520,4.267243,4.312280,...,4.286511,4.296638,4.335238,4.326571,4.348500,4.320823,4.335841,4.388642,4.349186,4.286329
4,4.003179,3.002774,2.878506,2.872646,2.880816,2.872736,2.845644,2.887444,2.866710,2.890623,...,2.881764,2.864815,2.851721,2.872886,2.864128,2.842840,2.861149,2.897097,2.875455,2.881974
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
938,4.259388,4.168138,4.224709,4.341040,4.281657,4.313750,4.250132,4.210386,4.951870,4.453033,...,4.271667,4.196898,4.277299,4.272368,4.280232,4.274329,4.251308,4.246295,4.240576,4.338556
939,3.482776,3.465223,3.409157,2.054939,3.507419,3.431127,3.985751,5.023639,3.016621,3.466193,...,3.469431,3.439416,3.464227,3.450730,3.470996,3.457788,3.446910,3.422900,3.458320,3.429034
940,4.964727,4.040018,4.056633,4.030073,4.080778,4.052060,4.050927,4.070411,4.046080,4.151706,...,3.968968,4.029242,4.057939,4.039843,4.048727,4.044439,4.038332,3.995543,4.047681,4.025192
941,4.234992,4.252155,4.241988,4.279108,4.251809,4.306603,4.248905,4.297231,4.279169,4.305345,...,4.227746,4.243324,4.294574,4.264360,4.284073,4.305989,4.281984,4.274143,4.273673,4.278289


In [35]:
preds_sps = preds_df.values
pivot_sps = pivot.values

In [36]:
#Для подсчета rmse были удалены NaN из изначального ДФ и элементы с такими же индексами из предсказаний
pivot_sps_1 = pivot_sps[~np.isnan(pivot_sps)]
preds_sps_1 = preds_sps[~np.isnan(pivot_sps)]

In [37]:
# mean_ = pivot_demeaned.values
# mean_ = mean_[~np.isnan(mean_)]
# mean_

In [38]:
# err = mse(mean_, preds_sps_1)
# rmse = sqrt(err)
# print(rmse)

In [39]:
from sklearn.metrics import mean_squared_error as mse
from math import sqrt

err = mse(pivot_sps_1, preds_sps_1)
rmse = sqrt(err)
print(rmse)

0.08907266816364375


In [40]:
pivot_sps_1

array([5., 3., 4., ..., 3., 3., 3.])

In [41]:
preds_sps_1

array([5.0004379 , 2.99465012, 4.01449725, ..., 2.99279481, 3.00502755,
       3.0207184 ])

In [42]:
mn = np.array(preds_df.mean(axis=1))
er = mse(users_mean_rating, mn)
rmse1 = sqrt(er)
print(rmse1)

4.002561121289517e-15


In [43]:
def recommend_movies(preds_df, userId, movie, original_ratings_df, num_recs):
    
    
    user_number = userId - 1 # UserID starts at 1, not 0
    sorted_user_predictions = preds_df.iloc[user_number].sort_values(ascending=False)
    
    user_data = original_ratings_df[original_ratings_df.userId == (userId)]
    user_data_merged = (user_data.merge(movie, how = 'left', left_on = 'movieId', right_on = 'movieId').
                     sort_values(['rating'], ascending=False))

    print (f'Пользователь {userId} оценил {user_data_merged.shape[0]} фильмов')
    print (f'Топ {num_recs} предсказанных фильмов:')
    
    recommendations = (movie[~movie['movieId'].isin(user_data_merged['movieId'])].
                    merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',
                    left_on = 'movieId', right_on = 'movieId').
                       rename(columns = {user_number: 'predictions'}).
                       sort_values('predictions', ascending = False).
                       iloc[:num_recs, :-1])

    return user_data_merged, recommendations

In [44]:
old_ratings, pred_ratings = recommend_movies(preds_df, 3, movie, data, 10)

Пользователь 3 оценил 54 фильмов
Топ 10 предсказанных фильмов:


In [45]:
old_ratings

,userId,movieId,rating,timestamp,title,date
53,3,346,5,889237455,Jackie Brown (1997),01-Jan-1997
52,3,340,5,889237455,Boogie Nights (1997),01-Jan-1997
49,3,347,5,889237455,Wag the Dog (1997),09-Jan-1998
8,3,328,5,889237455,Conspiracy Theory (1997),08-Aug-1997
31,3,320,5,889237482,Paradise Lost: The Child Murders at Robin Hood...,06-Dec-1996
26,3,321,5,889237455,Mother (1996),25-Dec-1996
17,3,348,4,889237455,Desperate Measures (1998),30-Jan-1998
45,3,344,4,889236939,"Apostle, The (1997)",18-Dec-1997
38,3,181,4,889237482,Return of the Jedi (1983),14-Mar-1997
36,3,329,4,889237455,Desperate Measures (1998),30-Jan-1998


In [46]:
pred_ratings

,movieId,title,date
855,910,Nil By Mouth (1997),06-Feb-1998
983,1038,Switchback (1997),01-Jan-1997
1015,1070,Living in Oblivion (1995),01-Jan-1995
810,865,"Ice Storm, The (1997)",01-Jan-1997
936,991,Keys to Tulsa (1997),11-Apr-1997
193,195,"Terminator, The (1984)",01-Jan-1984
503,558,Heavenly Creatures (1994),01-Jan-1994
833,888,One Night Stand (1997),01-Jan-1997
1082,1137,Beautiful Thing (1996),09-Oct-1996
1050,1105,Firestorm (1998),09-Jan-1998


In [24]:
preds_df.iloc[1][408]

3.822621397405612

In [25]:
preds_df.iloc[1][294]

1.873734826231528